In [1]:
import torch
from PerceiverAR import *
from dataset import *
from trainer import *

c:\Users\zhufe\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\zhufe\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\zhufe\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\Users\zhufe\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
BLOCK_SIZE = 128
LATENT_SIZE = 32
CORPUS_PATH = "wiki.txt"

In [3]:
text = open(CORPUS_PATH, encoding="utf-8").read()
perceiver_dataset = PerceiverARDataset(text, BLOCK_SIZE, LATENT_SIZE)

data has 418352 characters, 258 unique.


In [4]:
model_config = GPTConfig(
    perceiver_dataset.vocab_size,
    perceiver_dataset.block_size,
    n_layer=4,
    n_head=8,
    n_embd=256,
    perceiver=True,
    latent_size=perceiver_dataset.latent_size,
    padding_idx=perceiver_dataset.stoi[perceiver_dataset.PAD_CHAR],
)

model = PerceiverAR(model_config)
model.load_state_dict(torch.load("perceiver_ar.pt"))

<All keys matched successfully>

# Train

In [5]:
trainer_config = TrainerConfig(
    max_epochs=1,
    lr_decay=True,
    ckpt_path="perceiver_ar.pt",
)
trainer = Trainer(model, perceiver_dataset, trainer_config)

In [6]:
trainer.train()

epoch 1 iter 45: train loss 3.57202. lr 7.521280e-08: 100%|██████████| 46/46 [00:46<00:00,  1.01s/it]


# Test

In [7]:
TEST_INDEX = 200

In [8]:
def torch_to_string(x):
    return "".join([perceiver_dataset.itos[i] for i in x.numpy()])

In [9]:
x, y = perceiver_dataset[TEST_INDEX]
model.eval()
y_pred = model(x.unsqueeze(0))
max_indices = y_pred.argmax(dim=2)

In [16]:
x_string = torch_to_string(x)[:-LATENT_SIZE+1]
y_string = torch_to_string(y)
y_pred_string = torch_to_string(max_indices.squeeze(0))

In [17]:
print(f"Start - {x_string}")
print(f"Target - {y_string}")
print(f"Predicted - {y_pred_string}")

Start - □□□□□□□□□□□□□◎Douglas Fee. Douglas Fee (born 21 July 1944 in Kingston, Ontario) was a member of t
Target - he Canadian House of Commons fro
Predicted - oorain n nn a ro raraair  r  aon


In [12]:
# from torch.utils.data import DataLoader
# batch_size = 32
# data_loader = DataLoader(perceiver_dataset, batch_size=batch_size, shuffle=True)
# data_iter = iter(data_loader)
# x, y = next(data_iter)